# American Sign Language (ASL) Recognition using CNN approach

### This is a continuation of the American Sign Language (ASL) Recognition project where we have already used a MLP model to detect the gestures but it proved to be inefficient with a validation accuracy of 68.28%(overfitted model) which is not a very favourable score .

 ### So we will try to build a Convolutional Neural Network model  on the training dataset and compare its accuracy with the MLP model.

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import skimage
from skimage.transform import resize

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten
from keras.layers.normalization import BatchNormalization

### Importing dataset

In [ ]:
paths = r'C:\Users\KIIT\Documents\LGM-Soc contributions\American Sign Language (ASL) Recognition\Dataset/train/'
for dirname, _, filenames in os.walk(r'C:\Users\KIIT\Documents\LGM-Soc contributions\American Sign Language (ASL) Recognition\Dataset/train/'):
    print(dirname)

In [ ]:
categories = os.listdir(paths)
print("No. of categories of images in the train set = ",len(categories))

#### Since this is a supervised learning approach to train the model we will have to label the images before giving it as input to the CNN model


In [ ]:
def load_img_labels(categories):
    img_lst=[]
    labels=[]
    for index, category in enumerate(categories):
        n = 0
        for image_name in os.listdir(paths+"/"+category):
            if n==100:
                break
                
            #load image data into an array
            img = cv2.imread(paths+"/"+category+"/"+image_name)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_array = Image.fromarray(img, 'RGB')
            
             #resizing the image
            resized_img = img_array.resize((200, 200))
            
            #converting the image array to numpy array before appending it to the list
            img_lst.append(np.array(resized_img))
            
            #appending label
            labels.append(index)
            
            n+=1
    return img_lst, labels

In [ ]:
images, labels = load_img_labels(categories)
print("No. of images loaded = ",len(images),"\nNo. of labels loaded = ",len(labels))
print(type(images),type(labels))

#### Converting the images and labels into numpy arrays

In [ ]:
images = np.array(images)
labels = np.array(labels)

print("Images shape = ",images.shape,"\nLabels shape = ",labels.shape)
print(type(images),type(labels))

In [ ]:
def display_label_images(images, labels):
    plt.figure(1 , figsize = (15 , 10))
    n = 0 
    for i in range(4):
        n += 1 
        r = np.random.randint(0 , images.shape[0] , 1)
        
        plt.subplot(2, 2, n)
        plt.subplots_adjust(hspace = 0.3 , wspace = 0.1)
        plt.imshow(images[r[0]])
        
        plt.title('Assigned label : {}'.format(labels[r[0]]))
        plt.xticks([])
        plt.yticks([])
        
    plt.show()
    
display_label_images(images, labels)

### Generating X data and y data using Image augmentation approach

In [ ]:
batch_size = 64
imageSize = 64
target_dims = (imageSize, imageSize, 3)
num_classes = 29
train_len = 87000

In [ ]:
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']


In [ ]:
def data_generation(folder):
    X = np.empty((train_len, imageSize, imageSize, 3), dtype=np.float32)
    y = np.empty((train_len,), dtype=np.int)
    i = 0
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            for image_filename in os.listdir(folder + folderName):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file).reshape((-1, imageSize, imageSize, 3))
                    
                    X[i] = img_arr
                    y[i] = classes.index(folderName)
                    i += 1
    return X,y
X, y = data_generation(paths)

In [ ]:
print("X_data shape : ", X.shape)
print("y_data shape : ", y.shape)
print("Image shape : ", X[0].shape)

### Visualizing the X data --> sign language image and the y data --> label of the sign language

In [ ]:
f, axs = plt.subplots(2,2,figsize=(8,8))
for i in range (0,9):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    img_nbr = np.random.randint(0,len(X))
    plt.imshow(X[img_nbr])
    plt.title(classes[y[img_nbr]])

### Data Processing

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

#### Performing One Hot Encoding on the categorical features of the training data

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_encoded = to_categorical(y_train,29)
y_test_encoded = to_categorical(y_test,29)

### Verifying the dimensions of all variables before they are feeded to the models

In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

print(y_train_encoded.shape)
print(y_test_encoded.shape)

## Building the CNN model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten

In [ ]:
model = Sequential([
    Conv2D(32, (5, 5), input_shape=(64, 64, 3)),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    
    tf.keras.layers.Dropout(rate=0.3),
    
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dense(29, activation='softmax')
])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    X_train, y_train_encoded,
    epochs=50,
    batch_size=64,
    verbose=2,
    validation_data=(X_test, y_test_encoded),
    callbacks=[early_stop]
)

In [ ]:
history_df = pd.DataFrame(history.history)

#### Plotting the accuracy and validation accuracy

In [ ]:
history_df.loc[0:, ['accuracy','val_accuracy']].plot()
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training accuracy', 'validation accuracy'], loc='upper right')
plt.show()

#### Plotting the loss and validation loss

In [ ]:
history_df.loc[0:, ['loss', 'val_loss']].plot()
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training loss', 'validation loss'], loc='upper right')
plt.show()

### After training and testing the CNN model we can conclude that it reaches a training accuracy of 99.43% and validation accuracy of 99.78% which shows that the CNN model has gained a very high precision.

### Saving the CNN model

In [ ]:
model.save(r'C:\Users\KIIT\Documents\LGM-Soc contributions\American Sign Language (ASL) Recognition\Model/sign_language_Recognizer.model')

### Performing multiclass predictions

In [ ]:
predictions = np.argmax(model.predict(X_test), axis=-1)

In [ ]:
predictions

### Comparing the Actual label vs the Predicted label of the Sign Language image

In [ ]:
f, axs = plt.subplots(2,2,figsize=(15,15))
for i in range (0,25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    img_nbr = np.random.randint(0,len(predictions))
    plt.imshow(X[img_nbr])
    plt.title("True={} ; Predicted={} ".format(classes[(y_test[img_nbr])], classes[(predictions[img_nbr])]))

### Confusion Matrix 

In [ ]:
from sklearn.metrics import confusion_matrix
plt.figure(figsize=(8,8))
sns.heatmap(confusion_matrix(y_test,predictions))
plt.show()

### Classification report of the CNN model

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))